In [69]:
import torch
import numpy as np
import pandas as pd
import awkward
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
plt.rcParams.update({"font.size": 20})

import uproot
import vector
import glob
import tqdm
import os
import sys
import multiprocessing
from scipy.sparse import coo_matrix

import torch_geometric
from pyg.ssl.utils import data_split

In [4]:
! ls ../data/new_clic/

p8_ee_tt_ecm380


In [70]:
data_path = "../data/new_clic/"
data_split_mode = "mix"

In [7]:
data_VICReg_train, data_VICReg_valid, data_mlpf_train, data_mlpf_valid, data_test_qcd, data_test_ttbar = data_split(
        data_path, data_split_mode
    )

Will use data split mode `mix`
Will use 24235 events to train VICReg
Will use 2693 events to validate VICReg
Will use 6059 events to train MLPF
Will use 673 events to validate MLPF
Will use 3740 events to test MLPF on TTbar
Will use 0 events to test MLPF on QCD


In [8]:
data = data_VICReg_train+data_VICReg_valid+data_mlpf_train+data_mlpf_valid

In [75]:
loader = torch_geometric.loader.DataLoader(data, 1000000000000)
for events in loader:
    print("events contain all")

events contain all


In [76]:
mean = events.x[:,1:].mean(axis=0)
std = events.x[:,1:].std(axis=0)

In [99]:
mean

tensor([ 2.8273e+00, -2.9272e-04,  6.9944e-05,  6.2897e-03,  3.6280e+00,
         2.6983e+01,  5.6092e+00, -1.5506e+00,  1.0216e+00,  1.6612e+01,
         6.3377e-01,  5.2939e-04,  5.2478e+01,  3.2740e+01,  3.2399e+01,
         3.3631e+01])

In [98]:
std

tensor([1.1827e+01, 9.0163e-01, 7.0703e-01, 7.0716e-01, 3.7501e+01, 1.1569e+04,
        8.7224e+02, 1.2554e+03, 9.3771e-01, 3.9489e+01, 2.7251e+00, 1.7655e+00,
        9.3879e+01, 6.5164e+01, 6.2998e+01, 6.3626e+01])

In [102]:
# test standardization
loader = torch_geometric.loader.DataLoader(data, 200)
for batch in loader:
    print("Before standardization: \n" , batch.x)
    batch.x[:,1:] = (batch.x[:,1:] - mean)/std
    print("After standardization: \n" , batch.x)
    break

Before standardization: 
 tensor([[ 1.0000e+00,  4.2828e+01,  4.7899e-01,  ...,  4.6407e-03,
         -1.0000e+00,  0.0000e+00],
        [ 1.0000e+00,  2.3289e+01, -2.7211e-01,  ..., -1.3447e-03,
         -1.0000e+00,  0.0000e+00],
        [ 1.0000e+00,  1.0749e+01,  2.2698e-01,  ..., -4.3251e-03,
         -1.0000e+00,  0.0000e+00],
        ...,
        [ 2.0000e+00,  1.1334e+00,  4.4812e-01,  ...,  2.0008e+01,
          7.7585e+00,  1.2126e+01],
        [ 2.0000e+00,  5.2573e+00, -8.0025e-01,  ...,  6.1158e+02,
          8.0721e+02,  9.0094e+02],
        [ 2.0000e+00,  5.5991e+00, -4.4358e-01,  ...,  1.6386e+01,
          1.0624e+03,  4.8748e+02]])
After standardization: 
 tensor([[ 1.0000,  3.3821,  0.5316,  ..., -0.5024, -0.5302, -0.5286],
        [ 1.0000,  1.7301, -0.3015,  ..., -0.5024, -0.5302, -0.5286],
        [ 1.0000,  0.6698,  0.2521,  ..., -0.5025, -0.5302, -0.5286],
        ...,
        [ 2.0000, -0.1432,  0.4973,  ..., -0.1954, -0.3911, -0.3380],
        [ 2.0000,  0.205